In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import itertools

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
530,tt0271027,25000000,64437847,Kiss of the Dragon,Jet Li|Bridget Fonda|TchÃ©ky Karyo|Max Ryan|Ri...,Chris Nahon,Kiss Fear Goodbye,"Liu Jian, an elite Chinese police officer, com...",98,Action|Crime|Thriller,Twentieth Century Fox Film Corporation|Current...,7/6/2001,6.3,2001
12,tt0470752,15000000,36869414,Ex Machina,Domhnall Gleeson|Alicia Vikander|Oscar Isaac|S...,Alex Garland,There is nothing more human than the will to s...,"Caleb, a 26 year old coder at the world's larg...",108,Drama|Science Fiction,DNA Films|Universal Pictures International (UP...,1/21/2015,7.6,2015
730,tt1229238,145000000,694713380,Mission: Impossible - Ghost Protocol,Tom Cruise|Jeremy Renner|Simon Pegg|Paula Patt...,Brad Bird,No Plan. No Backup. No Choice.,In the 4th installment of the Mission Impossib...,133,Action|Thriller|Adventure,Paramount Pictures|Skydance Productions|Bad Ro...,12/7/2011,6.7,2011
1498,tt0409182,160000000,181674817,Poseidon,Kurt Russell|Richard Dreyfuss|Josh Lucas|Jacin...,Wolfgang Petersen,Mayday,A packed cruise ship traveling the Atlantic is...,99,Adventure|Action|Drama|Thriller,Virtual Studios|Next Entertainment|Warner Bros...,5/12/2006,5.3,2006
454,tt1055292,38000000,105610124,Life as We Know It,Katherine Heigl|Josh Duhamel|Christina Hendric...,Greg Berlanti,A comedy about taking it one step at a time.,Holly and Eric were set up on a blind date by ...,115,Comedy|Romance|Drama,Josephson Entertainment|Gold Circle Films|Vill...,10/8/2010,6.5,2010


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {}  # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

# the time given in the dataset is in string format.
# So we need to change this in datetime format
# ...

#import datetime
#data['release_date'] = data['release_date'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))
data['release_date'] = pd.to_datetime(data['release_date'])
data['profit'] = data['revenue'] - data['budget']
data['genres_array'] = data['genres'].str.split('|')
data['director_array'] = data['director'].str.split('|')
data['cast_array'] = data['cast'].str.split('|')
data['production_companies_array'] = data['production_companies'].str.split('|')
data['release_month_number'] = data['release_date'].dt.month
data['title_length'] = data['original_title'].apply(lambda x: len(x))
data['overview_words_count'] = data['overview'].apply(lambda x: len(x.split(' ')))
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,vote_average,release_year,profit,genres_array,director_array,cast_array,production_companies_array,release_month_number,title_length,overview_words_count
97,tt2279373,74000000,311594032,The SpongeBob Movie: Sponge Out of Water,Tom Kenny|Bill Fagerbakke|Rodger Bumpass|Clanc...,Paul Tibbitt,He's leaving his world behind.,Burger Beard is a pirate who is in search of t...,93,Family|Animation,...,5.7,2015,237594032,"[Family, Animation]",[Paul Tibbitt],"[Tom Kenny, Bill Fagerbakke, Rodger Bumpass, C...","[Paramount Pictures, Nickelodeon Movies, Nicke...",2,40,90
1350,tt0398375,70000000,88933562,Rumor Has It...,Jennifer Aniston|Kevin Costner|Shirley MacLain...,Rob Reiner,Based on a true rumor.,Sarah Huttinger's return home with her fiance ...,97,Comedy,...,5.2,2005,18933562,[Comedy],[Rob Reiner],"[Jennifer Aniston, Kevin Costner, Shirley MacL...","[Village Roadshow Pictures, Section Eight, Spr...",12,15,45
1805,tt0230030,48000000,90383208,Bedazzled,Elizabeth Hurley|Brendan Fraser|Frances O'Conn...,Harold Ramis,Meet the Devil. She's giving Elliott seven wis...,"Elliot Richardson, suicidal techno geek, is gi...",93,Fantasy|Comedy|Romance,...,5.3,2000,42383208,"[Fantasy, Comedy, Romance]",[Harold Ramis],"[Elizabeth Hurley, Brendan Fraser, Frances O'C...","[Twentieth Century Fox Film Corporation, Regen...",10,9,60
1856,tt0163978,40000000,144056873,The Beach,Leonardo DiCaprio|Guillaume Canet|Tilda Swinto...,Danny Boyle,Somewhere on this planet it must exist.,Twenty-something Richard travels to Thailand a...,119,Drama|Adventure|Romance|Thriller,...,6.2,2000,104056873,"[Drama, Adventure, Romance, Thriller]",[Danny Boyle],"[Leonardo DiCaprio, Guillaume Canet, Tilda Swi...","[20th Century Fox, Figment Films]",2,9,37
1311,tt1966566,30000000,68129518,Ð¡Ñ‚Ð°Ð»Ð¸Ð½Ð³Ñ€Ð°Ð´,Thomas Kretschmann|Yanina Studilina|Philippe R...,Fyodor Bondarchuk,The Epic Battle That Turned The Tide Of World ...,"Drama set in 1942, during one of the most impo...",131,War|Action,...,5.5,2013,38129518,"[War, Action]",[Fyodor Bondarchuk],"[Thomas Kretschmann, Yanina Studilina, Philipp...",[Art Pictures Studio],10,20,147


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример:
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [7]:
# тут пишем ваш код для решения данного вопроса:
line = data[data['budget'] == data['budget'].max()]
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [8]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [9]:
# думаю логику работы с этим словарем вы уже поняли,
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [10]:
line = data[data['runtime'] == data['runtime'].max()]
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [11]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [12]:
line = data[data['runtime'] == data['runtime'].min()]
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [13]:
answers['4'] = '2. 110'

In [14]:
int(round(data['runtime'].mean(), 0))

110

# 5. Каково медианное значение длительности фильмов? 

In [15]:
answers['5'] = '1. 107'

In [16]:
int(round(data['runtime'].agg(['median']), 0))

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [17]:
# лучше код получения столбца profit вынести в Предобработку что в начале
answers['6'] = '5. Avatar (tt0499549)'

In [18]:
line = data[data['profit'] == data['profit'].max()]
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [19]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [20]:
line = data[data['profit'] == data['profit'].min()]
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [21]:
answers['8'] = '1. 1478'

In [22]:
len(data[data['profit'] > 0])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [23]:
answers['9'] = '4. The Dark Knight (tt0468569)'

In [24]:
line_id = data.query('release_year == 2008')['revenue'].idxmax()
f"{data.iloc[line_id]['original_title']} ({data.iloc[line_id]['imdb_id']})"

'The Dark Knight (tt0468569)'

Вариант 2

In [25]:
line = data[data['release_year'] == 2008].sort_values('revenue', ascending=False)
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'The Dark Knight (tt0468569)'

Вариант 3

In [26]:
line = data.query('release_year == 2008').query('revenue == revenue.max()')
f"{line['original_title'].values[0]} ({line['imdb_id'].values[0]})"

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [27]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

In [28]:
line_id = data.query('release_year >= 2012 and release_year <= 2014')['profit'].idxmin()
f"{data.iloc[line_id]['original_title']} ({data.iloc[line_id]['imdb_id']})"

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [29]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

answers['11'] = '3. Drama'

In [30]:
data['genres_array'].explode().value_counts().head(1)

Drama    782
Name: genres_array, dtype: int64

Вариант 2

In [31]:
data['genres'].str.split('|', expand=True).stack().value_counts().head(1)

Drama    782
dtype: int64

Вариант 3

In [32]:
pd.Series(itertools.chain.from_iterable(data['genres_array'])).value_counts().head(1)

Drama    782
dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [33]:
answers['12'] = '1. Drama'

In [34]:
data.query('profit > 0')['genres_array'].explode().value_counts().head(1)

Drama    560
Name: genres_array, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [35]:
answers['13'] = '5. Peter Jackson'

In [36]:
data[['director_array', 'revenue']].explode('director_array').groupby('director_array').sum()\
    .sort_values('revenue', ascending=False).head(1)

,revenue
director_array,
Peter Jackson,6490593685


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [37]:
answers['14'] = '3. Robert Rodriguez'

Вариант 1

In [38]:
data[['director_array', 'genres_array']].explode('director_array').explode('genres_array')\
    .query('genres_array == "Action"')['director_array'].value_counts().head(1)

Robert Rodriguez    9
Name: director_array, dtype: int64

Вариант 2

In [39]:
data[data['genres'].str.contains(
    'Action')]['director_array'].explode().value_counts().head(1)

Robert Rodriguez    9
Name: director_array, dtype: int64

Вариант 3

In [40]:
directors = Counter(data[data['genres'].str.contains(
    'Action')]['director_array'].explode())
directors.most_common(1)

[('Robert Rodriguez', 9)]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [41]:
answers['15'] = '3. Chris Hemsworth'

In [42]:
data.query('release_year == 2012')[['cast_array', 'revenue']].explode('cast_array').groupby('cast_array').sum()\
    .sort_values('revenue', ascending=False).head(1)

,revenue
cast_array,
Chris Hemsworth,2027450773


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [43]:
answers['16'] = '3. Matt Damon'

In [44]:
mean_budget = data['budget'].mean()
data[data['budget'] > mean_budget]['cast_array'].explode().value_counts().head(1)

Matt Damon    18
Name: cast_array, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [45]:
answers['17'] = '2. Action'

Вариант 1

In [46]:
data[data['cast'].str.contains(
    'Nicolas Cage')]['genres_array'].explode().value_counts().head(1)

Action    17
Name: genres_array, dtype: int64

Вариант 2 (с защитой от актеров с похожим именем, например Nicolas Cagesss)

In [47]:
data[['cast_array', 'genres_array']].explode('cast_array').query(
    'cast_array == "Nicolas Cage"')['genres_array'].explode().value_counts().head(1)

Action    17
Name: genres_array, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [48]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

In [49]:
line_id = data[['production_companies_array', 'profit']].explode('production_companies_array').query(
    'production_companies_array == "Paramount Pictures"')['profit'].idxmin()
f"{data.iloc[line_id]['original_title']} ({data.iloc[line_id]['imdb_id']})"

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [50]:
answers['19'] = '5. 2015'

In [51]:
data[['release_year', 'revenue']].groupby('release_year').sum()\
    .sort_values('release_year', ascending=False).head(1)

,revenue
release_year,
2015,25449202382


# 20. Какой самый прибыльный год для студии Warner Bros?

In [52]:
answers['20'] = '1. 2014'

In [53]:
data[['production_companies', 'profit', 'release_year']].query(
    'production_companies.str.contains("Warner Bros")', engine='python').groupby(
    'release_year').sum().sort_values('profit', ascending=False).head(1)

,profit
release_year,
2014,2295464519


Вариант 2
index.unique() нужно для защиты от дубля из-за строчки "Warner Bros.|Warner Bros. Interactive Entertainment|Rob Thomas Productions".

In [54]:
indexes = data[['production_companies_array', 'profit', 'release_year']].explode('production_companies_array').query(
    'production_companies_array.str.contains("Warner Bros")', engine='python').index.unique()
data.iloc[indexes][['profit', 'release_year']].groupby(
    'release_year').sum().sort_values('profit', ascending=False).head(1)

,profit
release_year,
2014,2295464519


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [55]:
answers['21'] = '4. Сентябрь'

In [78]:
data['release_month_number'].value_counts().head(1)

9    227
Name: release_month_number, dtype: int64

Вариант 2

In [57]:
data[['release_date']].groupby(data['release_date'].dt.month).agg(
    count=('release_date', 'count')).sort_values('count', ascending=False).head(1)

,count
release_date,
9,227


Вариант 3

In [58]:
data[['release_date']].groupby(data['release_date'].dt.month).size().reset_index(
    name='counts').sort_values('counts', ascending=False).head(1)

,release_date,counts
8,9,227


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [59]:
answers['22'] = '2. 450'

In [60]:
len(data.query('release_month_number in (6,7,8)'))

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [61]:
answers['23'] = '5. Peter Jackson'

In [62]:
data.query('release_month_number in (12,1,2)')['director_array'].reset_index().explode('director_array').groupby(
    'director_array').size().reset_index(name='count').sort_values('count', ascending=False).head(1)

,director_array,count
265,Peter Jackson,7


Вариант 2

In [80]:
ds = []
for items in data.query('release_month_number in (12,1,2)')['director_array']:
    ds.extend(items)
Counter(ds).most_common(1)

[('Peter Jackson', 7)]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [64]:
answers['24'] = '5. Four By Two Productions'

In [65]:
data[['production_companies_array', 'title_length']].explode('production_companies_array').groupby(
    'production_companies_array').mean().sort_values('title_length', ascending=False).head(1)

,title_length
production_companies_array,
Four By Two Productions,83.0


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [66]:
answers['25'] = '3. Midnight Picture Show'

In [67]:
data[['production_companies_array', 'overview_words_count']].explode('production_companies_array').groupby(
    'production_companies_array').mean().sort_values('overview_words_count', ascending=False).head(1)

,overview_words_count
production_companies_array,
Midnight Picture Show,175.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [68]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [69]:
data[data['vote_average'] >= data['vote_average'].quantile(q=0.99)]['original_title'].sort_values()

1191                                     12 Years a Slave
316                                              3 Idiots
124                                            Big Hero 6
1213                                   Dallas Buyers Club
1563                Eternal Sunshine of the Spotless Mind
128                                             Gone Girl
119                               Guardians of the Galaxy
1188                                                  Her
370                                             Inception
9                                              Inside Out
118                                          Interstellar
1800                                              Memento
283                                            Mr. Nobody
1205                                            Prisoners
34                                                   Room
28                                              Spotlight
599                                       The Dark Knight
177           

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [70]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

In [71]:
pairs = {}
for casts in data['cast_array']:
    casts = sorted(casts)
    for num, c in enumerate(casts):
        for i in range(num+1, len(casts)):
            key = f'{c} & {casts[i]}'
            pairs.setdefault(key, 0)
            pairs[key] += 1

pd.DataFrame(pairs.items(), columns=['pair', 'count']).sort_values('count', ascending=False).head(10)

,pair,count
2395,Daniel Radcliffe & Emma Watson,8
2397,Daniel Radcliffe & Rupert Grint,8
2400,Emma Watson & Rupert Grint,8
3686,Helena Bonham Carter & Johnny Depp,6
1290,Ben Stiller & Owen Wilson,6
49,Paul Walker & Vin Diesel,5
2554,Robert Pattinson & Taylor Lautner,5
2553,Kristen Stewart & Taylor Lautner,5
2552,Kristen Stewart & Robert Pattinson,5
131,Adam Sandler & Kevin James,5


Вариант 2

In [72]:
pairs = {}
for casts in data['cast_array']:
    for pair in itertools.combinations(casts, 2):
        key = ' & '.join(sorted(pair))
        pairs.setdefault(key, 0)
        pairs[key] += 1
        if key == 'Rupert Grint & Emma Watson':
            print(casts)

pd.DataFrame(pairs.items(), columns=['pair', 'count']).sort_values('count', ascending=False).head(10)

,pair,count
2399,Emma Watson & Rupert Grint,8
2395,Daniel Radcliffe & Rupert Grint,8
2396,Daniel Radcliffe & Emma Watson,8
3684,Helena Bonham Carter & Johnny Depp,6
1293,Ben Stiller & Owen Wilson,6
1307,Hugh Jackman & Ian McKellen,5
40,Paul Walker & Vin Diesel,5
2545,Kristen Stewart & Robert Pattinson,5
2546,Kristen Stewart & Taylor Lautner,5
2549,Robert Pattinson & Taylor Lautner,5


# Submission

In [73]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [74]:
# и убедиться что ни чего не пропустил)
len(answers)

27